In [4]:
from pathlib import Path
import os
import json

DATA_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/datasets/klevr_nesf")
MODEL_BASE_DIR = Path("/data/vision/polina/projects/wmh/dhollidt/documents/nerf/klever_models_nesf")

TRAIN_START = 0
TEST_START = 150

TIMESTAMP = "17_03_23"

def get_highest_load_step(load_dir: Path):
    load_step = sorted(int(x[x.find("-") + 1 : x.find(".")]) for x in os.listdir(load_dir))[-1]
    return load_step
    

def create_nesf_data_config(num_scenes: int, split: str, out_path: Path):
    start = TRAIN_START if split == "train" else TEST_START
    
    
    config = []
    for scene_idx in range(start, start+num_scenes):
        load_dir = MODEL_BASE_DIR / str(scene_idx) / "nerfacto" / TIMESTAMP / "nerfstudio_models"
        scene_config = {
            "model_config": {},
            "data_parser_config": {
                "data": str(DATA_BASE_DIR / str(scene_idx)),
            },
            "load_step": get_highest_load_step(load_dir),
            "load_dir": str(load_dir),
            }
        
        if split == "test":
            scene_config["data_parser_config"]["train_split_percentage"] = 0.0
            scene_config["set_type"] = "test"
        
        config.append(scene_config)
        
    data_config = {"config": config}
    
    # write to file
    with open(out_path, 'w') as f:
        json.dump(data_config, f, indent=4)
    
    

In [7]:
create_nesf_data_config(10, "train", Path("/data/vision/polina/projects/wmh/dhollidt/documents/nerf/data/klever_nesf_train_10.json"))
create_nesf_data_config(10, "test", Path("/data/vision/polina/projects/wmh/dhollidt/documents/nerf/data/klever_nesf_test_10.json"))